In [1]:
import numpy as np
import pandas as pd
import glob
import re

In [2]:
datadir = "../data/McGill-Billboard/"
# filepaths = glob.glob(datadir+'[0-9][0-9][0-9][0-9]/salami_chords.txt')
filepaths = glob.glob(datadir+'1256/salami_chords.txt')

In [75]:
fileidx = 0
filepath = filepaths[fileidx]

def ExtractFile(filepath):
    f = open(filepath, 'r')
    raw = f.readlines()
    f.close
    return raw

def ExtractMeta(raw):
    title = re.split('# title: |\n', raw[0])[1]
    artist = re.split('# artist: |\n', raw[1])[1]
    metre = re.split('# metre: |\n', raw[2])[1]
    tonic = re.split('# tonic: |\n', raw[3])[1]

    return (title, artist, metre, tonic)

raw = ExtractFile(filepath)
(title, artist, metre, tonic) = ExtractMeta(raw)

In [76]:
def ExtractPhrases(raw):
    raw = raw[6:-2]
    phrases = list()
    for row in raw:
        splitted = re.split('\t', row)[1:][0]
        splitted = re.split('\|', splitted)[1:]
        
        if not splitted: continue
        
        phrase = splitted[:-1]

        # check for repeat
        if IsRepeat(splitted[-1][1:3]):
            phrase.append(splitted[-1][1:3])

        phrases.append(phrase)
    return phrases

def CleanPhrases(phrases):
    phrases_clean = list()
    for phrase in phrases:
        phrase_clean = [chord.strip() for chord in phrase]
        phrases_clean.append(phrase_clean)
    return phrases_clean
    
def FormatChords(phrases_clean):
    chords = list()
    
    for phrase in phrases_clean:
        # clean non-chords
        phrase_chord = [RemoveNoise(chord) for chord in phrase]
        
        # remove blank (noise-only) lines
        if all([not p for p in phrase_chord]): continue

        # check for repeat
        if IsRepeat(phrase_chord[-1]):
            repeated = phrase_chord[:-1]
            n_repeat = int(phrase_chord[-1][-1])
            phrase_chord = repeated*n_repeat

        chords.append(phrase_chord)
    # flatten 2d list
    chords = sum(chords, [])

    # split chords into bars
    chords = [chord.split() for chord in chords]

    return chords

def IsRepeat(str):
    if re.match(r'x\d', str):
        return True
    else:
        return False

def RemoveNoise(chord):
    chord_clean = re.sub(r'\.|\*|\(\d+/\d+\)|N', '', chord)
    chord_clean = chord_clean.split()
    chord_clean = ' '.join(chord_clean)
    return chord_clean

phrases = ExtractPhrases(raw)
phrases_clean = CleanPhrases(phrases)
chords = FormatChords(phrases_clean)

In [77]:
def CreateIndex(chords):
    indexs_bar = list()
    for i, bar in enumerate(chords):
        index_bar = [i]*len(bar)
        indexs_bar.append(index_bar)
    return indexs_bar
chords_index = CreateIndex(chords)

In [78]:
def Raw2Basic(chord_raw):
    root = chord_raw.split(':')[0]
    func = chord_raw.split(':')[1].split('/')[0]
    
    if func[:3]=='maj':
        func = 'maj'
    elif func[:3]=='min':
        func = 'min'
    elif func[:3]=='sus':
        func = func[:4]
    elif func[0]=='1' or func[0]=='5':
        func = '15'
    
    chord_basic = root+':'+func
    return chord_basic

def Raw2Root(chord_raw):
    root = chord_raw.split(':')[0]
    return root

In [80]:
def GetEnharmonic(note):
    if note == 'Db':
        return 'C#'
    elif note == 'Eb':
        return 'D#'
    elif note == 'E#':
        return 'F'
    elif note == 'Fb':
        return 'E'
    elif note == 'Gb':
        return 'F#'
    elif note == 'Ab':
        return 'G#'
    elif note == 'Bb':
        return 'A#'
    elif note == 'B#':
        return 'C'
    elif note == 'Cb':
        return 'B'
    return note

def Abs2RelRoot(chord_root, tonic):
    chromatic = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    relative = ['I', 'I#', 'II', 'II#', 'III', 'IV', 'IV#', 'V', 'V#', 'VI', 'VI#', 'VII']

    tonic = GetEnharmonic(tonic)
    chord_root = GetEnharmonic(chord_root)

    tonic_pos = chromatic.index(tonic)
    chord_root_pos = chromatic.index(chord_root)

    dist = chord_root_pos - tonic_pos
    rel_chord_root = relative[dist]

    return rel_chord_root

def Abs2Rel(chord, tonic):
    splitted = chord.split(':')
    root = splitted[0]
    func = ''
    if len(splitted) > 1:
        func = ':' + splitted[1]
    rel_root = Abs2RelRoot(root, tonic)
    rel_chord = rel_root + func

    return rel_chord

Abs2Rel('Bb:maj7', 'Ab')

'II:maj7'

In [91]:
def Rel2InCRoot(root_rel, tonic='C'):
    chromatic = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    relative = ['I', 'I#', 'II', 'II#', 'III', 'IV', 'IV#', 'V', 'V#', 'VI', 'VI#', 'VII']

    chromatic = chromatic[chromatic.index(tonic):] + chromatic[:chromatic.index(tonic)]
    root_inc = chromatic[relative.index(root_rel)]

    return root_inc

def Rel2InC(chord_rel, tonic='C'):
    splitted = chord_rel.split(':')
    root = splitted[0]
    func = ''
    if len(splitted) > 1:
        func = ':' + splitted[1]
    rel_root = Rel2InCRoot(root, tonic)
    rel_chord = rel_root + func

    return rel_chord

In [103]:
df_chords = pd.DataFrame()
df_chords['bar'] = sum(chords_index, [])
df_chords['chords'] = sum(chords, [])
df_chords['chords_basic'] = df_chords['chords'].apply(Raw2Basic)
df_chords['chords_root'] = df_chords['chords'].apply(Raw2Root)

df_chords['chords_rel'] = df_chords['chords'].apply(Abs2Rel, tonic=tonic)
df_chords['chords_basic_rel'] = df_chords['chords_basic'].apply(Abs2Rel, tonic=tonic)
df_chords['chords_root_rel'] = df_chords['chords_root'].apply(Abs2Rel, tonic=tonic)

df_chords['chords_inC'] = df_chords['chords_rel'].apply(Rel2InC)
df_chords['chords_basic_inC'] = df_chords['chords_basic_rel'].apply(Rel2InC)
df_chords['chords_root_inC'] = df_chords['chords_root_rel'].apply(Rel2InC)

df_chords['meta'] = ['title: '+title, 'artist: '+artist, 'metre: '+metre, 'tonic: '+tonic] + [np.nan] * (len(df_chords)-4)

In [104]:
df_chords

,bar,chords,chords_basic,chords_root,chords_rel,chords_basic_rel,chords_root_rel,chords_inC,chords_basic_inC,chords_root_inC
0,0,C:1,C:15,C,I:1,I:15,I,C:1,C:15,C
1,1,C:1,C:15,C,I:1,I:15,I,C:1,C:15,C
2,2,C:1,C:15,C,I:1,I:15,I,C:1,C:15,C
3,3,C:1,C:15,C,I:1,I:15,I,C:1,C:15,C
4,4,C:min7,C:min,C,I:min7,I:min,I,C:min7,C:min,C
...,...,...,...,...,...,...,...,...,...,...
302,202,G:sus4(b7),G:sus4,G,V:sus4(b7),V:sus4,V,G:sus4(b7),G:sus4,G
303,203,G:sus4(b7),G:sus4,G,V:sus4(b7),V:sus4,V,G:sus4(b7),G:sus4,G
304,204,G:sus4(b7),G:sus4,G,V:sus4(b7),V:sus4,V,G:sus4(b7),G:sus4,G
305,205,Bb:maj/9,Bb:maj,Bb,VI#:maj/9,VI#:maj,VI#,A#:maj/9,A#:maj,A#
